In [1]:
import numpy as np
import nibabel as nib
from dipy.data import fetch_taiwan_ntu_dsi, read_taiwan_ntu_dsi, get_sphere
from dipy.reconst.gqi import GeneralizedQSamplingModel
from dipy.direction import peaks_from_model
from dipy.viz import window, actor, fvtk
from dipy.segment.mask import median_otsu
from dipy.io import read_bvals_bvecs
from dipy.core.gradients import gradient_table
from dipy.io.image import save_nifti
from dipy.io.image import save_nifti

In [2]:
fetch_taiwan_ntu_dsi()
img, gtab = read_taiwan_ntu_dsi()

Data size is approximately 91MB
Dataset is already in place. If you want to fetch it again please first remove the folder /homes/9/cy281/.dipy/taiwan_ntu_dsi 
See DSI203_license.txt for LICENSE. For the complete datasets please visit :          http://dsi-studio.labsolver.org
Data size is approximately 91MB
Dataset is already in place. If you want to fetch it again please first remove the folder /homes/9/cy281/.dipy/taiwan_ntu_dsi 
See DSI203_license.txt for LICENSE. For the complete datasets please visit :          http://dsi-studio.labsolver.org


In [5]:
data = img.get_data()
print('data.shape (%d, %d, %d, %d)' % data.shape)
voxel_size = img.header.get_zooms()[:3]
print(gtab.bvals.shape)
print(gtab.bvecs.shape)

data.shape (96, 96, 60, 203)
(203,)
(203, 3)


In [6]:
maskdata, mask = median_otsu(data, 3, 1, False,
                             vol_idx=range(10, 50), dilate=2)

In [28]:
sphere = get_sphere('symmetric724')

gqmodel = GeneralizedQSamplingModel(gtab, 'standard', sampling_length=1.2)

gqpeaks = peaks_from_model(model=gqmodel, data=data,
                           sphere=sphere, mask=mask,
                           relative_peak_threshold=.5,
                           min_separation_angle=25,
                           return_odf=True,
                           normalize_peaks=True)

# sphere = get_sphere('symmetric724')
# gqpeaks = peaks_from_model(model=gqmodel, data=data,
#                            sphere=sphere, mask=mask,
#                            relative_peak_threshold=.5,
#                            min_separation_angle=25,
#                            return_odf=True, return_sh=True, sh_basis_type='mrtrix',
#                            normalize_peaks=True)

# # save odf
# nib.save(nib.Nifti1Image(gqpeaks.shm_coeff.astype(np.float32),
#                                   img.affine, img.header), '/space/hemera/1/users/cmaffei/scilpy_mrtrix_comparison/gqtest_dipy_data/odf_sh_mrtrix_12.nii.gz')


In [18]:
#Enables/disables interactive visualization
interactive = True
ren = window.Renderer()
window.clear(ren)
gq_peaks = actor.peak_slicer(gqpeaks.peak_dirs, gqpeaks.peak_values, colors=None)
ren.add(gq_peaks)

if interactive:
    window.show(ren)

In [32]:
import matplotlib.pyplot as plt 
vox_odf = gqpeaks.odf[58, 49, 39, :]
vox_odf.shape
plt.plot(vox_odf)
plt.title('GQI2 Method')
plt.savefig('/space/hemera/1/users/cmaffei/scilpy_mrtrix_comparison/gqtest_dipy_data/gqi1_2_vox58,49,39.png')
plt.show()

(724,)

In [29]:
#Enables/disables interactive visualization
interactive = True
r = window.Renderer()
gqiodfs = gqpeaks.odf

# gqiodfs = np.clip(gqiodfs, 0, np.max(gqiodfs, -1)[..., None])
gqi_odfs_actor = actor.odf_slicer(gqiodfs, sphere=sphere, scale=0.5, colormap='plasma')
gqi_odfs_actor.display_extent(58, 58, 49, 49, 39, 39)
r.set_camera(position=(0, 10, 0))
# gqi_odfs_actor.display(z=0)
r.add(gqi_odfs_actor)
window.record(r, out_path='/space/hemera/1/users/cmaffei/scilpy_mrtrix_comparison/gqtest_dipy_data/odf_standard_vox58,49,39.png', size=(600, 600))
if interactive:
        window.show(r)

In [50]:
from dipy.direction import (peaks_from_model, ProbabilisticDirectionGetter)
from dipy.data import small_sphere

pmf = gqpeaks.odf.clip(min=0)
prob_dg = ProbabilisticDirectionGetter.from_pmf(pmf, max_angle=30, sphere=sphere)

In [61]:
GFA = gqpeaks.gfa

print('GFA.shape (%d, %d, %d)' % GFA.shape)

GFA.shape (96, 96, 60)


In [64]:
from dipy.tracking import utils
from dipy.tracking.local import (ThresholdTissueClassifier, LocalTracking,ActTissueClassifier)
from dipy.tracking.streamline import Streamlines
from dipy.io.streamline import save_trk
from dipy.tracking.utils import random_seeds_from_mask

seeds = random_seeds_from_mask(GFA > 0.4, seeds_count=1)
tissue_classifier = ThresholdTissueClassifier(GFA, .25)

In [16]:
#Probabilistic Tractography
streamlines_generator = LocalTracking(prob_dg, tissue_classifier, seeds, affine=np.eye(4), step_size=0.5)
#Generate streamline object
streamlines = Streamlines(streamlines_generator)

#Save the trk file
save_trk("/space/hemera/1/users/cmaffei/scilpy_mrtrix_comparison/gqtest_dipy_data/probabilistic_gqtest.trk",
         streamlines,
         img.affine, shape=img.shape[:3], vox_size=voxel_size)

#Deterministic Tractography
streamline_generator = LocalTracking(
        gqpeaks, tissue_classifier, seeds, affine=np.eye(4),
        step_size=0.5)

streamlines = Streamlines(streamline_generator)
save_trk("/space/hemera/1/users/cmaffei/scilpy_mrtrix_comparison/gqtest_dipy_data/deterministic_gqtest.trk",
         streamlines,
         img.affine, shape=img.shape[:3], vox_size=voxel_size)

print("Number of streamlines: " + str(len(streamlines)))

Number of streamlines: 61078
